In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from matplotlib.colors import is_color_like as color_check
import requests
import random
import pickle
import re
import spacy
nlp = spacy.load('en_core_web_trf')
from spacy import displacy
import collections
from collections import Counter
from tqdm.notebook import tqdm as tqdm_notebook
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from transformers import DistilBertTokenizer, DistilBertModel, logging
from matplotlib.figure import Figure
from matplotlib import cm
import matplotlib.colors as colors
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
# URL
URL = 'https://en.wikipedia.org/wiki/Glossary_of_plant_morphology'
# Get the page
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, "lxml", from_encoding="iso-8859-1")   

glossary = collections.defaultdict(list)
# Find all H4 
for chapter in soup.find_all('h4')[0:]:
    # Clean
    chapter_text = chapter.text.rstrip('[edit]')
    # Find all siblings
    for sibling in chapter.find_next_siblings():
        # Find the parent
        for parent in sibling.find_previous_sibling('h4'):
            # Only append if correspond to current chapter
            if parent.text == chapter_text:
                if 'â' in sibling.text:
                    for tag in sibling.find_all('li'):
                        candidates = tag.text.split('â')[0]
                        candidates = candidates.split('/')
                        for candidate in candidates:
                            glossary[chapter_text.lower()].append(candidate.strip().lower())  
                            
glossary['leaves'] += [
    'glume', 'surface', 'margin',
    'leave', 'leaf', 'auricles', 'spatheole',
    'ovate', 'lanceolate',
]

glossary['basic flower parts'] += [
    'floret', 'awn',
    'pod', 'lobe', 
    'capitulum', 'capitula', # unkown
    'legume', 'calyx', 'flowerhead'
]
glossary['inflorescences'] += [
    'spikelets', 'lemma', 'racemes',
    'axis', 'cluster', 
]
glossary['leaves'] += [
    'rhachilla',
    'needles',
]

glossary['other parts'] += [
    'apex', 'culm', 'tube',
    'palea', 'crown', 'canopy',
    'base', 'callus', 'hair',
    'anther', 'tuberculate', 'cone',
    'shoot', 'gland',

]

glossary['plant property'] += [
    'tree', 'shrub', 'plant',
    'life-span', 'life', 'span',
    'bloom-time', 'species', 'wood', 'timber',
    'color', 'colour', 
    
]

glossary['stems'] += [
    'branchlet', 
    
]

In [3]:
glossary.keys()

dict_keys(['morphology', 'roots', 'stems', 'buds', 'leaves', 'basic flower parts', 'inflorescences', 'insertion of floral parts', 'union of flower parts', 'flower sexuality and presence of floral parts', 'flower symmetry', 'terms for fruits', 'fruit types', 'pteridophytes', 'bryophytes', 'other parts', 'plant property'])

In [23]:
Parts = pickle.load(open('../../data/glossaries/FNA_parts.pkl', 'rb'))
toplevel = pickle.load(open('../../data/glossaries/FNA_toplevels.pkl', 'rb'))

In [24]:
glossary_FNA = collections.defaultdict(list)

for key in Parts:
    glossary_FNA[key] += Parts[key]
    
for key in toplevel:
    glossary_FNA[key] += toplevel[key]

In [45]:
glossary_FNA['Flower']

['1-locular anther fusion',
 'Abaxial and adaxial scale architecture or function or pubescence',
 'Abaxial and adaxial scale quantity',
 'Abaxial bristle length',
 'Abaxial bristle quantity',
 'Abaxial calyx architecture or shape',
 'Abaxial calyx some measurement',
 'Abaxial outer tepal coloration',
 'Abaxial perianth coloration',
 'Abaxial petal atypical length',
 'Abaxial petal atypical width',
 'Abaxial petal height or length or size',
 'Abaxial petal length',
 'Abaxial petal length or size',
 'Abaxial petal prominence',
 'Abaxial petal size',
 'Abaxial petal some measurement',
 'Abaxial petal width',
 'Abaxial scale architecture or arrangement',
 'Abaxial scale architecture or pubescence or shape',
 'Abaxial scale architecture or shape',
 'Abaxial scale arrangement',
 'Abaxial scale arrangement or course or shape',
 'Abaxial scale length',
 'Abaxial scale presence',
 'Abaxial scale prominence',
 'Abaxial scale quantity',
 'Abaxial scale shape',
 'Abaxial scale size',
 'Abaxial sca

In [9]:
#glossary_FNA['Flower']

In [10]:
compounds = [
    'fertile', 'sterile',
    'male', 'female', 'bisexual',
    'basal', 'developed', 
    'primary', 'secondary', 'main',
    'upper', 'lower', 'greater', 'dorsal', 'alternate', 'lesser', 'apex', 'outer',
    'central', 'outermost', 'outer', 'inner', 'uppermost', 'median', 'dorsal', 'central', 'lateral',
    'young', 'mature', 'individual', 
    'opposite', 
]

rubbish = [
    '.', ',', '-', '..', '...',
]

measurements = [
    'mm', 'cm', 'm', 'km',
    'milimeter', 'centimeter', 'meter', 'kilometer',
    'milimetre', 'centimetre', 'metre', 'kilometre',
    'inch', 'foot', 'yard', 'mile',
    'wide', 'long', 'broad', 'tall',
    'length', 'form',
]


In [11]:
data = pickle.load(open('../../data/PlantNet/descriptions_raw.pkl', 'rb'))
#data = pickle.load(open('../../data/description/04_TRAIN_0000000-0014557_PLANTS.pkl', 'rb'))

In [27]:
data = {
    'Test Tree':[
        '5 slightly dark, light-green leaf without any blue veins.',
        'Stems often 5-veined.',
        'Branches light-dark reddish brown and greenish purple or black.'
    ]
}

In [28]:


measurements_list = [
    'mm', 'cm', 'm', 'km',
    'milimeter', 'centimeter', 'meter', 'kilometer',
    'milimetre', 'centimetre', 'metre', 'kilometre',
    'inch', 'foot', 'yard', 'mile',
    'wide', 'long', 'broad', 'tall',
    #'length', 'form',
]

In [29]:
def text_preparation(species, text):
    cleaners = [(r'(?<!\d)\.(?!\d)', ' '),
                (r'\s×\s', ' times '),
                #(r'\xa0', ' '),
                (r'\s+c\s+', ' '),
                (r'â\x80\x93', ' to '),
                (r'\xa0', ' '),
                (r'\x97', ''),
                (r'\s{2,}', ' ')]
    
    species_parts = species.split()
    candidates = [' '.join(species_parts[:idx+1]) for idx, _ in enumerate(species_parts)]
    candidates += [
        f'{species_parts[0][0]}. {species_parts[1]}'
    ]
    candidates.reverse()
    for candidate in candidates:
        try:
            text = re.sub(candidate, 'the species', text)
        except:
            continue # Skip species with brackets for now
    for (cleaner, replacement) in cleaners:
        text = re.sub(cleaner, replacement, text)    
    text = f'{text.strip()}.'
    return text.capitalize()

In [30]:
def extract_modifiers(t, doc):
    # Init list
    modifiers_dict = {}
    
    # AMODS
    amods = extract_amods(t, doc)
    for child in amods:
        amods = extract_modifiers(child, doc)
        modifiers_dict[child] = amods
        
    # APPOS
    compounds = extract_compounds(t, doc)
    for child in compounds:
        compounds = extract_modifiers(child, doc)
        modifiers_dict[child] = compounds
        
    # APPOS
    appos = extract_appos(t, doc)
    for child in appos:
        appos = extract_modifiers(child, doc)
        modifiers_dict[child] = appos
        
    # Preposition
    prep = extract_preps(t, doc)
    for child in prep:
        prep = extract_modifiers(child, doc)
        modifiers_dict[child] = prep
    
    # Preposition Objects
    prep_objs = extract_prep_ojects(t, doc)
    for child in prep_objs:
        prep_objs = extract_modifiers(child, doc)
        modifiers_dict[child] = prep_objs
        
    # MEASUREMENTS
    measurements = extract_measurements(t, doc)
    for child in measurements:
        measurements = extract_modifiers(child, doc)
        modifiers_dict[child] = measurements 
        
    # MEASUREMENTS UNITS
    units = extract_units(t, doc)
    for child in units:
        units = extract_modifiers(child, doc)
        modifiers_dict[child] = units
        
    units_extra = extract_units_modifiers(t, doc)
    for child in units_extra:
        units_extra = extract_modifiers(child, doc)
        modifiers_dict[child] = units_extra
     
    # MEASUREMENTS UNITS MAX
    units_max = extract_units_max(t, doc)
    for child in units_max:
        units_max = extract_modifiers(child, doc)
        modifiers_dict[child] = units_max
        
        # MEASUREMENTS UNITS MIN
        units_min = extract_units_min(child, doc)
        for child in units_min:
            units_min = extract_modifiers(child, doc)
            modifiers_dict[child] = units_min     
       
    return modifiers_dict

def extract_amods(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'amod'
           and tok.lemma_ not in measurements_list]

def extract_compounds(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'compound'
           and tok.lemma_ not in measurements_list]

def extract_appos(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'appos'
           and tok.lemma_ not in measurements_list]

def extract_preps(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'prep'
           and tok.lemma_ not in measurements_list]

def extract_prep_ojects(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'pobj'
           and tok.lemma_ not in measurements_list]

def extract_measurements(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'amod'
           and tok.lemma_ in measurements_list]

def extract_units(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'npadvmod'
           and tok.lemma_ in measurements_list]

def extract_units_modifiers(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'advmod'
           and tok.pos_ == 'NUM']

def extract_units_max(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'nummod']

def extract_units_min(t, doc):
    return [tok for tok in t.children if tok.dep_ == 'quantmod']




In [38]:
def check_existance(t):
    
    return next((key for key, value in glossary.items() if t.lemma_.lower() in value), None)
    

def extract_candidates(doc):
    
    return [t for t in doc 
            if t.pos_ == 'NOUN' 
            or t.pos_ == 'PROPN' 
            or t.pos_ == 'PRON']


def undict_to_pairs(d):
    for k, v in d.items():
        if isinstance(v, dict):
            for subk in v:
                yield (k, subk)
            yield from undict_to_pairs(v)
        #else:
        #    yield (k, v)
                
                
def extract_conjunctions(t):
    for tok in t.children:
        if tok.dep_ == 'conj':
            yield (tok)
        yield from extract_conjunctions(tok)

def extract_information(species, doc):
    
    triples = []
        
    doubles = collections.defaultdict(list)   
        
    for candidate in extract_candidates(doc): 
        existance = check_existance(candidate)
        
        
        if existance:
            information = {candidate: extract_modifiers(candidate, doc)}
            
            # Keep conjunctions outside of the main recursion
            candidate_conjunctions = extract_conjunctions(candidate)        
            for candidate_conjunction in candidate_conjunctions:
                information[candidate][candidate_conjunction] = extract_modifiers(candidate_conjunction, doc)
            #doubles[(existance, candidate)] = list(undict_to_pairs(information))
            print((existance, candidate), *list(undict_to_pairs(information)))
            #print(list(undict_to_pairs(information)))
            doubles[species].append([(existance, candidate), *list(undict_to_pairs(information))])
        #print(information)
        
    #return list(set(doubles))
    return doubles
        
    #return doubles
    #return doub

In [39]:
descriptions = collections.defaultdict(list)

for species in tqdm_notebook(list(data.keys())[0:1]):
    for idx, text in enumerate(data[species][0:1]):
        text = text_preparation(species, text)
        doc = nlp(text)
        doubles = extract_information(species, doc)
        

            
            
                   
                    
        #print(text)
        #print(idx, triples)
        #print('\n')
        #descriptions[species] += triples    

  0%|          | 0/1 [00:00<?, ?it/s]

('stems', leaf) (leaf, dark) (leaf, green) (leaf, without) (leaf, 5) (green, light) (without, veins) (veins, blue)
('leaves', veins) (veins, blue)


In [36]:
doubles

defaultdict(list, {})

In [37]:
displacy.render(doc)

In [ ]:
def extract_conjunctions(doc):
    for t in doc.children:
        if t.dep_ == 'conj':
            yield (t)
        yield from extract_conjunctions(t)

In [ ]:
list(extract_conjunctions(doc[5]))

In [ ]:
doc[5]

In [ ]:
doc

In [ ]:
x

In [ ]:
print(len(glossary.keys()))

In [ ]:
count = 0
for k, v in glossary.items():
    count += len(k)
    count += len(v)

In [ ]:
count